<a href="https://colab.research.google.com/github/Prangfa/MyGit_sec02/blob/main/Copy_of_Lab06_Constraint_Satisfaction_Problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRAINT SATISFACTION PROBLEMS

This IPy notebook acts as supporting material for topics covered in **Chapter 6 Constraint Satisfaction Problems** of the book* Artificial Intelligence: A Modern Approach*. We make use of the implementations in **csp.py** module. Even though this notebook includes a brief summary of the main topics, familiarity with the material present in the book is expected. We will look at some visualizations and solve some of the CSP problems described in the book. Let us import everything from the csp module to get started.

In [165]:
!git clone -l -s https://github.com/aimacode/aima-python.git aima

fatal: destination path 'aima' already exists and is not an empty directory.


In [166]:
!cp aima/utils.py utils.py
!cp aima/search.py search.py
!cp aima/notebook.py notebook.py
!cp aima/games.py games.py
!cp aima/leaustralia_cspobabilistic_learning.py probabilistic_learning.py
!cp aima/logic.py logic.py
!cp aima/agents.py agents.py
!cp aima/csp.py csp.py
!cp aima/learning.py learning.py
!cp aima/probabilistic_learning.py probabilistic_learning.py
!cp -r /content/aima/images /content/images

cp: cannot stat 'aima/leaustralia_cspobabilistic_learning.py': No such file or directory


In [167]:
!mkdir aima-data
!wget https://github.com/aimacode/aima-data/raw/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/orings.csv -P /content/aima-data/
!wget https://github.com/aimacode/aima-data/raw/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/zoo.csv -P /content/aima-data/
!wget https://github.com/aimacode/aima-data/raw/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/iris.csv -P /content/aima-data/
!wget https://github.com/aimacode/aima-data/raw/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/restaurant.csv -P /content/aima-data/
!pip install qpsolvers
!pip install ipythonblocks

mkdir: cannot create directory ‘aima-data’: File exists
--2024-07-25 13:31:32--  https://github.com/aimacode/aima-data/raw/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/orings.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aimacode/aima-data/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/orings.csv [following]
--2024-07-25 13:31:32--  https://raw.githubusercontent.com/aimacode/aima-data/f6cbea61ad0c21c6b7be826d17af5a8d3a7c2c86/orings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405 [text/plain]
Saving to: ‘/content/aima-data/orings.csv.4’

orings.csv.4        100%[===================>]     405 

In [168]:
from csp import *
from notebook import psource, plot_NQueens
%matplotlib inline

# Hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

## CONTENTS

* Overview
* Graph Coloring
* N-Queens
* Backtracking Search
* Graph Coloring Visualization
* N-Queens Visualization

## OVERVIEW

CSPs are a special kind of search problems. Here we don't treat the space as a black box but the state has a particular form and we use that to our advantage to tweak our algorithms to be more suited to the problems. A CSP State is defined by a set of variables which can take values from corresponding domains. These variables can take only certain values in their domains to satisfy the constraints. A set of assignments which satisfies all constraints passes the goal test. Let us start by exploring the CSP class which we will use to model our CSPs.

In [169]:
psource(CSP)

The __ _ _init_ _ __ method parameters specify the CSP. Variables can be passed as a list of strings or integers. Domains are passed as dict (dictionary datatpye) where "key" specifies the variables and "value" specifies the domains. The variables are passed as an empty list. Variables are extracted from the keys of the domain dictionary. Neighbor is a dict of variables that essentially describes the constraint graph. Here each variable key has a list of its values which are the variables that are constraint along with it. The constraint parameter should be a function **f(A, a, B, b**) that **returns true** if neighbors A, B **satisfy the constraint** when they have values **A=a, B=b**. We have additional parameters like nassings which is incremented each time an assignment is made when calling the assign method. You can read more about the methods and parameters in the class doc string. We will talk more about them as we encounter their use. Let us jump to an example.

## GRAPH COLORING

We use the graph coloring problem as our running example for demonstrating the different algorithms in the **csp module**. The idea of map coloring problem is that the adjacent nodes (those connected by edges) should not have the same color throughout the graph. The graph can be colored using a fixed number of colors. Here each node is a variable and the values are the colors that can be assigned to them. Given that the domain will be the same for all our nodes we use a custom dict defined by the **UniversalDict** class. The **UniversalDict** Class takes in a parameter and returns it as a value for all the keys of the dict. It is very similar to **defaultdict** in Python except that it does not support item assignment.

In [170]:
s = UniversalDict(['R','G','B'])
s[5]

['R', 'G', 'B']

For our CSP we also need to define a constraint function **f(A, a, B, b)**. In this, we need to ensure that the neighbors don't have the same color. This is defined in the function **different_values_constraint** of the module.

In [171]:
psource(different_values_constraint)

The CSP class takes neighbors in the form of a Dict. The module specifies a simple helper function named **parse_neighbors** which allows us to take input in the form of strings and return a Dict of a form that is compatible with the **CSP Class**.

In [172]:
%pdoc parse_neighbors

The **MapColoringCSP** function creates and returns a CSP with the above constraint function and states. The variables are the keys of the neighbors dict and the constraint is the one specified by the **different_values_constratint** function. **Australia**, **USA** and **France** are three CSPs that have been created using **MapColoringCSP**. **Australia** corresponds to ** Figure 6.1 ** in the book.

In [173]:
psource(MapColoringCSP)

In [174]:
australia_csp, usa_csp, france_csp

(<csp.CSP at 0x7d64441a6110>,
 <csp.CSP at 0x7d64441a5ff0>)

## N-QUEENS

The N-queens puzzle is the problem of placing N chess queens on an N×N chessboard in a way such that no two queens threaten each other. Here N is a natural number. Like the graph coloring problem, NQueens is also implemented in the csp module. The **NQueensCSP** class inherits from the **CSP** class. It makes some modifications in the methods to suit this particular problem. The queens are assumed to be placed one per column, from left to right. That means position (x, y) represents (var, val) in the CSP. The constraint that needs to be passed to the CSP is defined in the **queen_constraint** function. The constraint is satisfied (true) if A, B are really the same variable, or if they are not in the same row, down diagonal, or up diagonal.

In [175]:
psource(queen_constraint)

The **NQueensCSP** method implements methods that support solving the problem via **min_conflicts** which is one of the many popular techniques for solving CSPs. Because **min_conflicts** hill climbs the number of conflicts to solve, the CSP **assign** and **unassign** are modified to record conflicts. More details about the structures: **rows**, **downs**, **ups** which help in recording conflicts are explained in the docstring.

In [176]:
psource(NQueensCSP)

The _ ___init___ _ method takes only one parameter **n** i.e. the size of the problem. To create an instance, we just pass the required value of n into the constructor.

In [177]:
eight_queens = NQueensCSP(8)

### Helper Functions

We will now implement a few helper functions that will allow us to visualize the Coloring Problem; we'll also make a few modifications to the existing classes and functions for additional record keeping. To begin, we modify the **assign** and **unassign** methods in the **CSP** in order to add a copy of the assignment to the **assignment_history**. We name this new class as **InstruCSP**; it will allow us to see how the assignment evolves over time.

In [178]:
import copy
class InstruCSP(CSP):
    def __init__(self, variables, domains, neighbors, constraints):
        super().__init__(variables, domains, neighbors, constraints)
        self.assignment_history = []
        self.conflict_count = 0

    def assign(self, var, val, assignment):
        super().assign(var,val, assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
        self.conflict_count += self.count_conflicts(var, val, assignment)

    def unassign(self, var, assignment):
        super().unassign(var,assignment)
        self.assignment_history.append(copy.deepcopy(assignment))

    def count_conflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""
        count = 0
        for var2 in self.neighbors[var]:
            if var2 in assignment and not self.constraints(var, val, var2, assignment[var2]):
                count += 1
        return count

Next, we define **make_instru** which takes an instance of **CSP** and returns an instance of **InstruCSP**.

In [179]:
def make_instru(csp):
    return InstruCSP(csp.variables, csp.domains, csp.neighbors, csp.constraints)

We will now use a graph defined as a dictionary for plotting purposes in our Graph Coloring Problem. The keys are the nodes and their values are the corresponding nodes they are connected to.

In [180]:
neighbors = {
    0: [6, 11, 15, 18, 4, 11, 6, 15, 18, 4],
    1: [12, 12, 14, 14],
    2: [17, 6, 11, 6, 11, 10, 17, 14, 10, 14],
    3: [20, 8, 19, 12, 20, 19, 8, 12],
    4: [11, 0, 18, 5, 18, 5, 11, 0],
    5: [4, 4],
    6: [8, 15, 0, 11, 2, 14, 8, 11, 15, 2, 0, 14],
    7: [13, 16, 13, 16],
    8: [19, 15, 6, 14, 12, 3, 6, 15, 19, 12, 3, 14],
    9: [20, 15, 19, 16, 15, 19, 20, 16],
    10: [17, 11, 2, 11, 17, 2],
    11: [6, 0, 4, 10, 2, 6, 2, 0, 10, 4],
    12: [8, 3, 8, 14, 1, 3, 1, 14],
    13: [7, 15, 18, 15, 16, 7, 18, 16],
    14: [8, 6, 2, 12, 1, 8, 6, 2, 1, 12],
    15: [8, 6, 16, 13, 18, 0, 6, 8, 19, 9, 0, 19, 13, 18, 9, 16],
    16: [7, 15, 13, 9, 7, 13, 15, 9],
    17: [10, 2, 2, 10],
    18: [15, 0, 13, 4, 0, 15, 13, 4],
    19: [20, 8, 15, 9, 15, 8, 3, 20, 3, 9],
    20: [3, 19, 9, 19, 3, 9]
}

Now we are ready to create an InstruCSP instance for our problem. We are doing this for an instance of **MapColoringProblem** class which inherits from the **CSP** Class. This means that our **make_instru** function will work perfectly for it.

In [181]:
coloring_problem = MapColoringCSP('RGBY', neighbors)

In [182]:
coloring_problem1 = make_instru(coloring_problem)

## BACKTRACKING SEARCH

The main issue with using Naive Search Algorithms to solve a CSP is that they can continue to expand obviously wrong paths; whereas, in **backtracking search**, we check the constraints as we go and we deal with only one variable at a time. Backtracking Search is implemented in the repository as the function **backtracking_search**. This is the same as **Figure 6.5** in the book. The function takes as input a CSP and a few other optional parameters which can be used to speed it up further. The function returns the correct assignment if it satisfies the goal. However, we will discuss these later. For now, let us solve our **coloring_problem1** with **backtracking_search**.

In [183]:
psource(backtracking_search)

In [205]:
result = backtracking_search(coloring_problem1)

In [185]:
result # A dictonary of assignments.

{0: 'R',
 1: 'R',
 2: 'R',
 3: 'R',
 4: 'G',
 5: 'R',
 6: 'G',
 7: 'R',
 8: 'B',
 9: 'R',
 10: 'G',
 11: 'B',
 12: 'G',
 13: 'G',
 14: 'Y',
 15: 'Y',
 16: 'B',
 17: 'B',
 18: 'B',
 19: 'G',
 20: 'B'}

Let us also check the number of assignments made.

In [186]:
coloring_problem1.nassigns

21

Now, let us check the total number of assignments and unassignments, which would be the length of our assignment history. We can see it by using the command below.

In [187]:
len(coloring_problem1.assignment_history)

21

Now let us explore the optional keyword arguments that the **backtracking_search** function takes. These optional arguments help speed up the assignment further. Along with these, we will also point out the methods in the CSP class that help to make this work.

The first one is **select_unassigned_variable**. It takes in, as a parameter, a function that helps in deciding the order in which the variables will be selected for assignment. We use a heuristic called Most Restricted Variable which is implemented by the function **mrv**. The idea behind **mrv** is to choose the variable with the least legal values left in its domain. The intuition behind selecting the **mrv** or the most constrained variable is that it allows us to encounter failure quickly before going too deep into a tree if we have selected a wrong step before. The **mrv** implementation makes use of another function **num_legal_values** to sort out the variables by the number of legal values left in its domain. This function, in turn, calls the **nconflicts** method of the **CSP** to return such values.

In [188]:
psource(mrv)

In [189]:
psource(num_legal_values)

In [190]:
psource(CSP.nconflicts)

Another ordering related parameter **order_domain_values** governs the value ordering. Here we select the Least Constraining Value which is implemented by the function **lcv**. The idea is to select the value which rules out least number of values in the remaining variables. The intuition behind selecting the **lcv** is that it allows a lot of freedom to assign values later. The idea behind selecting the mrc and lcv makes sense because we need to do all variables but for values, and it's better to try the ones that are likely. So for vars, we face the hard ones first.

In [191]:
psource(lcv)

Finally, the third parameter **inference** can make use of one of the two techniques called Arc Consistency or Forward Checking. The details of these methods can be found in the **Section 6.3.2** of the book. In short the idea of inference is to detect the possible failure before it occurs and to look ahead to not make mistakes. **mac** and **forward_checking** implement these two techniques. The **CSP** methods **support_pruning**, **suppose**, **prune**, **choices**, **infer_assignment** and **restore** help in using these techniques. You can find out more about these by looking up the source code.

Now let us compare the performance with these parameters enabled vs the default parameters. We will use the Graph Coloring problem instance 'usa' for comparison. We will call the instances **solve_simple** and **solve_parameters** and solve them using backtracking and compare the number of assignments.

In [192]:
solve_simple = copy.deepcopy(usa_csp)
solve_parameters = copy.deepcopy(usa_csp)

In [193]:
backtracking_search(solve_simple)
backtracking_search(solve_parameters, order_domain_values=lcv, select_unassigned_variable=mrv, inference=mac)

{'ID': 'R',
 'WY': 'G',
 'UT': 'B',
 'MT': 'B',
 'CO': 'R',
 'NV': 'G',
 'AZ': 'R',
 'SD': 'R',
 'ND': 'G',
 'OR': 'B',
 'CA': 'Y',
 'NE': 'B',
 'IA': 'G',
 'MO': 'R',
 'WA': 'G',
 'IL': 'B',
 'MN': 'B',
 'KA': 'G',
 'KY': 'G',
 'WI': 'R',
 'OK': 'B',
 'TN': 'B',
 'NM': 'G',
 'VA': 'R',
 'TX': 'R',
 'WV': 'B',
 'AR': 'G',
 'LA': 'B',
 'IN': 'R',
 'OH': 'Y',
 'MD': 'G',
 'PA': 'R',
 'DE': 'B',
 'NJ': 'G',
 'DC': 'B',
 'NC': 'G',
 'MI': 'G',
 'NY': 'B',
 'MS': 'R',
 'AL': 'G',
 'GA': 'R',
 'FL': 'B',
 'SC': 'B',
 'CT': 'R',
 'MA': 'G',
 'RI': 'B',
 'VT': 'R',
 'NH': 'B',
 'ME': 'R'}

In [194]:
solve_simple.nassigns

49

In [195]:
solve_parameters.nassigns

49

## GRAPH COLORING VISUALIZATION

Next, we define some functions to create the visualisation from the assignment_history of **coloring_problem1**. The readers need not concern themselves with the code that immediately follows as it is the usage of Matplotib with IPython Widgets. If you are interested in reading more about these, visit [ipywidgets.readthedocs.io](http://ipywidgets.readthedocs.io). We will be using the **networkx** library to generate graphs. These graphs can be treated as graphs that need to be colored or as constraint graphs for this problem. If interested you can check out a fairly simple tutorial [here](https://www.udacity.com/wiki/creating-network-graphs-with-python). We start by importing the necessary libraries and initializing matplotlib inline.


In [196]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import time

The ipython widgets we will be using require the plots in the form of a step function such that there is a graph corresponding to each value. We define the **make_update_step_function** which returns such a function. It takes in as inputs the neighbors/graph along with an instance of the **InstruCSP**. The example below will elaborate it further. If this sounds confusing, don't worry. This is not part of the core material and our only goal is to help you visualize how the process works.

In [197]:
def make_update_step_function(graph, instru_csp):

    #define a function to draw the graphs
    def draw_graph(graph):

        G=nx.Graph(graph)
        pos = nx.spring_layout(G,k=0.15)
        return (G, pos)

    G, pos = draw_graph(graph)
    def map_color(color):
        if color == 'R':
            return 'red'
        elif color == 'G':
            return 'green'
        elif color == 'B':
            return 'blue'
        elif color == 'Y':
            return 'yellow'
        else:
            return 'black'

    def update_step(iteration):
        # here iteration is the index of the assignment_history we want to visualize.
        current = instru_csp.assignment_history[iteration]
        #  We convert the particular assignment to a default dict so that the color for nodes which
        # have not been assigned defaults to black.
        current = defaultdict(lambda: 'Black', current)

        # Now we use colors in the list and default to black otherwise.
        colors = [map_color(current[node]) for node in G.nodes.keys()]
        # Finally drawing the nodes.
        nx.draw(G, pos, node_color=colors, node_size=500)

        labels = {label:label for label in G.nodes}
        # Labels shifted by offset so that nodes don't overlap
        label_pos = {key:[value[0], value[1]+0.03] for key, value in pos.items()}
        nx.draw_networkx_labels(G, label_pos, labels, font_size=20)

        # display the graph
        plt.show()

    return update_step  # <-- this is a function

def make_visualize(slider):
    ''' Takes an input a slider and returns
        callback function for timer and animation
    '''

    def visualize_callback(Visualize, time_step):
        if Visualize is True:
            for i in range(slider.min, slider.max + 1):
                slider.value = i
                time.sleep(float(time_step))

    return visualize_callback


Finally let us plot our problem. We first use the function below to obtain a step function.

In [198]:
step_func = make_update_step_function(neighbors, coloring_problem1)

Next, we set the canvas size.

In [199]:
matplotlib.rcParams['figure.figsize'] = (18.0, 18.0)

Finally, our plot using ipywidget slider and matplotib. You can move the slider to experiment and see the colors change. It is also possible to move the slider using arrow keys or to jump to the value by directly editing the number with a double click. The **Visualize Button** will automatically animate the slider for you. The **Extra Delay Box** allows you to set time delay in seconds (upto one second) for each time step.

In [200]:
import ipywidgets as widgets
from IPython.display import display

iteration_slider = widgets.IntSlider(min=0, max=len(coloring_problem1.assignment_history)-1, step=1, value=0)
w=widgets.interactive(step_func,iteration=iteration_slider)
display(w)

visualize_callback = make_visualize(iteration_slider)

visualize_button = widgets.ToggleButton(description = "Visualize", value = False)
time_select = widgets.ToggleButtons(description='Extra Delay:',options=['0', '0.1', '0.2', '0.5', '0.7', '1.0'])

a = widgets.interactive(visualize_callback, Visualize = visualize_button, time_step=time_select)
display(a)

interactive(children=(IntSlider(value=0, description='iteration', max=20), Output()), _dom_classes=('widget-in…

## N-QUEENS VISUALIZATION

Just like the Graph Coloring Problem, we will start with defining a few helper functions to help us visualize the assignments as they evolve over time. The **make_plot_board_step_function** behaves similar to the **make_update_step_function** introduced earlier. It initializes a chess board in the form of a 2D grid with alternating 0s and 1s. This is used by **plot_board_step** function which draws the board using matplotlib and adds queens to it. This function also calls the **label_queen_conflicts** which modifies the grid placing a 3 in any position where there is a conflict.

In [ ]:
def label_queen_conflicts(assignment,grid):
    ''' Mark grid with queens that are under conflict. '''
    for col, row in assignment.items(): # check each queen for conflict
        conflicts = {temp_col:temp_row for temp_col,temp_row in assignment.items()
                         if (temp_row == row and temp_col != col)
                          or (temp_row+temp_col == row+col and temp_col != col)
                          or (temp_row-temp_col == row-col and temp_col != col)}

        # Place a 3 in positions where this is a conflict
        for col, row in conflicts.items():
                grid[col][row] = 3

    return grid

def make_plot_board_step_function(instru_csp):
    '''ipywidgets interactive function supports
       single parameter as input. This function
       creates and return such a function by taking
       in input other parameters.
    '''
    n = len(instru_csp.variables)


    def plot_board_step(iteration):
        ''' Add Queens to the Board.'''
        data = instru_csp.assignment_history[iteration]

        grid = [[(col+row+1)%2 for col in range(n)] for row in range(n)]
        grid = label_queen_conflicts(data, grid) # Update grid with conflict labels.

        # color map of fixed colors
        cmap = matplotlib.colors.ListedColormap(['white','lightsteelblue','red'])
        bounds=[0,1,2,3] # 0 for white 1 for black 2 onwards for conflict labels (red).
        norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

        fig = plt.imshow(grid, interpolation='nearest', cmap = cmap,norm=norm)

        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

        # Place the Queens Unicode Symbol
        for col, row in data.items():
            fig.axes.text(row, col, u"\u265B", va='center', ha='center', family='Dejavu Sans', fontsize=32)
        plt.show()

    return plot_board_step

Now let us visualize a solution obtained via backtracking. We make use of the previosuly defined **make_instru** function for keeping a history of steps.

In [ ]:
twelve_queens_csp = NQueensCSP(12)
backtracking_instru_queen = make_instru(twelve_queens_csp)
result = backtracking_search(backtracking_instru_queen)

In [ ]:
backtrack_queen_step = make_plot_board_step_function(backtracking_instru_queen) # Step Function for Widgets

Now finally we set some matplotlib parameters to adjust how our plot will look like. The font is necessary because the Black Queen Unicode character is not a part of all fonts. You can move the slider to experiment and observe how the queens are assigned. It is also possible to move the slider using arrow keys or to jump to the value by directly editing the number with a double click. The **Visualize Button** will automatically animate the slider for you. The **Extra Delay Box** allows you to set time delay in seconds of upto one second for each time step.

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)
matplotlib.rcParams['font.family'].append(u'Dejavu Sans')

iteration_slider = widgets.IntSlider(min=0, max=len(backtracking_instru_queen.assignment_history)-1, step=0, value=0)
w=widgets.interactive(backtrack_queen_step,iteration=iteration_slider)
display(w)

visualize_callback = make_visualize(iteration_slider)

visualize_button = widgets.ToggleButton(description = "Visualize", value = False)
time_select = widgets.ToggleButtons(description='Extra Delay:',options=['0', '0.1', '0.2', '0.5', '0.7', '1.0'])

a = widgets.interactive(visualize_callback, Visualize = visualize_button, time_step=time_select)
display(a)

# Lab

แก้ไขโค้ดเพื่อนับจำนวนข้อขัดแย้ง (conflict) ที่เกิดขึ้นระหว่างการค้นหาแบบ Backtracking และแสดงผลออกมา

In [124]:
import copy


def make_instru(csp):
    return InstruCSP(csp.variables, csp.domains, csp.neighbors, csp.constraints)

australia_csp = make_instru(australia_csp)
usa_csp = make_instru(usa_csp)
france_csp = make_instru(france_csp)
twelve_queens_csp = make_instru(NQueensCSP(12))


result_australia = backtracking_search(australia_csp)
result_usa = backtracking_search(usa_csp)
result_france = backtracking_search(france_csp)
result_twelve_queens = backtracking_search(twelve_queens_csp)

print(f"Australia CSP: {australia_csp.conflict_count} conflicts")
print(f"USA CSP: {usa_csp.conflict_count} conflicts")
print(f"France CSP: {france_csp.conflict_count} conflicts")
print(f"Twelve Queens CSP: {twelve_queens_csp.conflict_count} conflicts")

Australia CSP: 0 conflicts
USA CSP: 0 conflicts
France CSP: 0 conflicts
Twelve Queens CSP: 0 conflicts
